In [2]:
# Dependencies and Setup
import pandas as pd


# Raw data file
file_to_load = "Resources/nfl_data.csv"

# Read purchasing file and store into pandas data frame
nfldata_raw = pd.read_csv(file_to_load)

In [3]:
#Grabbing Columns relavent to our research
nfl = nfldata_raw[["Date", "GameID", "down", "Yards.Gained", "FirstDown", "Touchdown", "PlayType", \
                  "PassOutcome", "PassLength", "AirYards", "PassLocation", "RushAttempt", \
                  "RunLocation", "Reception", "Season"]]

#creating a column to count play for future sumations
nfl["PlayCount"] = 1

#dropping rows where [down]=Nan; these represent special teams plays and extra points
nfl.dropna(subset=["down"], inplace=True)

C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Kristopher Bognot\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
#Blank Lists used to append
ptotals = []
p = []
rtotals = []
r = []

In [5]:
#NFL Passing Data
#Grabbing only the records where the PlayType = Pass from our NFL Data
nfl_pass = nfl.loc[nfl['PlayType'] == "Pass"]

#Create DataFrame for only Passing columns
npass = nfl_pass[["down", "Yards.Gained", "AirYards", "FirstDown", "Touchdown", "PlayCount"]]

#Group by Down
p = npass.groupby(["down"]).sum()

#Create DataFrame of Totals for all downs
ptotals = pd.DataFrame({"down": ["Totals"], 
                       "Yards.Gained": p["Yards.Gained"].sum(),
                       "AirYards": p["AirYards"].sum(),
                       "FirstDown": p["FirstDown"].sum(),
                       "Touchdown": p["Touchdown"].sum(),
                       "PlayCount": p["PlayCount"].sum()})
ptotals = ptotals.set_index("down")
#Append Totals to Pass table
p = p.append(ptotals)

#Average/Rates
p['AvgYardsGained_PerPlay'] = round(p["Yards.Gained"] / p["PlayCount"], 3)
p['RateofFirstDown_PerPlay'] = round(p["FirstDown"] / p["PlayCount"], 3)
p['RateofTouchdown_PerPlay'] = round(p["Touchdown"] / p["PlayCount"], 3)

p

,Yards.Gained,AirYards,FirstDown,Touchdown,PlayCount,AvgYardsGained_PerPlay,RateofFirstDown_PerPlay,RateofTouchdown_PerPlay
down,,,,,,,,
1.0,447898,512854,16030.0,2613,59329,7.549,0.270,0.044
2.0,367297,402372,16295.0,2419,52840,6.951,0.308,0.046
3.0,312234,387925,15568.0,2379,44229,7.059,0.352,0.054
4.0,15725,26602,2098.0,238,2530,6.215,0.829,0.094
Totals,1143154,1329753,49991.0,7649,158928,7.193,0.315,0.048


In [13]:
#Pass Breakdowns

#Create DataFrame for only Passing columns
npass = nfl_pass[["down", "Yards.Gained", "AirYards", "FirstDown", "Touchdown", "PlayCount", "PassLocation", "PassOutcome"]]
npass["PassCompletedCount"] =  

if npass["PassOutcome"] == "Complete":
    =1
    Else 0

#Group by Down
pb = npass.groupby(["down", "PassLocation"]).sum()
pb

SyntaxError: invalid syntax (<ipython-input-13-075dbeb75192>, line 5)

In [6]:
#NFL Rushing Data
#Grabbing only the records where the PlayType = Pass from our NFL Data
nfl_run = nfl.loc[nfl['PlayType'] == "Run"]

#Create DataFrame for only Passing columns
nrun = nfl_run[["down", "Yards.Gained", "RushAttempt", "FirstDown", "Touchdown", "PlayCount"]]

#Group by Down
r = nrun.groupby(["down"]).sum()

#Create DataFrame of Totals for all downs
rtotals = pd.DataFrame({"down": ["Totals"], 
                       "Yards.Gained": r["Yards.Gained"].sum(),
                       "RushAttempt": r["RushAttempt"].sum(),
                       "FirstDown": r["FirstDown"].sum(),
                       "Touchdown": r["Touchdown"].sum(),
                       "PlayCount": r["PlayCount"].sum()})
rtotals = rtotals.set_index("down")
#Append Totals to Pass table
r = r.append(rtotals)

#Average/Rates
r['AvgYardsGained_PerPlay'] = round(r["Yards.Gained"] / r["PlayCount"], 3)
r['RateofFirstDown_PerPlay'] = round(r["FirstDown"] / r["PlayCount"], 3)
r['RateofTouchdown_PerPlay'] = round(r["Touchdown"] / r["PlayCount"], 3)

r

,Yards.Gained,RushAttempt,FirstDown,Touchdown,PlayCount,AvgYardsGained_PerPlay,RateofFirstDown_PerPlay,RateofTouchdown_PerPlay
down,,,,,,,,
1.0,285943,65549,7248.0,1739,65549,4.362,0.111,0.027
2.0,177949,40838,10388.0,1266,40838,4.357,0.254,0.031
3.0,58603,12638,5390.0,619,12638,4.637,0.426,0.049
4.0,5300,1599,1319.0,139,1599,3.315,0.825,0.087
Totals,527795,120624,24345.0,3763,120624,4.376,0.202,0.031
